<a href="https://colab.research.google.com/github/Saikrishnaraju0510/bigquery/blob/main/batch_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-beam[gcp]

     |████████████████████████████████| 9.9 MB 5.7 MB/s 
     |████████████████████████████████| 2.3 MB 57.1 MB/s 
     |████████████████████████████████| 247 kB 72.7 MB/s 
     |████████████████████████████████| 151 kB 71.4 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 173 kB 47.0 MB/s 
     |████████████████████████████████| 435 kB 50.0 MB/s 
     |████████████████████████████████| 180 kB 53.7 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 171 kB 51.8 MB/s 
     |████████████████████████████████| 183 kB 58.2 MB/s 
     |████████████████████████████████| 110 kB 62.8 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 255 kB 60.3 MB/s 
     |████████████████████████████████| 267 kB 53.7 MB/s 
     |████████████████████████████████| 109 kB 73.5 MB/s 
     |███████████████

In [ ]:
!pip install google-apitools

In [ ]:
from apache_beam.io.gcp.bigquery import WriteToBigQuery
import argparse
import os
from datetime import date
from google.cloud import bigquery
path1='modular-asset-332406-b566e60f8085.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path1
tab_id='airflow1.cashpayments'
tab_id2='airflow1.walletpayments'
tab_id1='airflow1.cardpayments'
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

parser = argparse.ArgumentParser()
# parser.add_argument('--my-arg')
args, beam_args = parser.parse_known_args()

# Create and set your Pipeline Options.
beam_options = PipelineOptions(
    beam_args,
   # runner='DataflowRunner',
    temp_location='gs://us-central1-c1-dea909ea-bucket/temp1',
    service_account_email='demo-py@modular-asset-332406.iam.gserviceaccount.com',
    region='us-central1',
    project='modular-asset-332406',
    stage_location='gs://sai_1999/stage'
    )
#args = beam_options.view_as(MyOptions)
#today= 2548566
client = bigquery.Client()

dataset_id = "modular-asset-332406.airflow1"

try:
	client.get_dataset(dataset_id)

except:
	dataset = bigquery.Dataset(dataset_id)  #

	dataset.location = "US"
	dataset.description = "dataset"

	dataset_ref = client.create_dataset(dataset)
def to_json(fields):
    json_str = {"Invoice_ID":fields[0],
                 "Branch": fields[1],
                 "City": fields[2],
                 "Customer_type": fields[3],
                 "Gender": fields[4],
                 "Product_line": fields[5],
                 "Unit_price": fields[6],
                 "Quantity": fields[7],
                 "Tax_5": fields[8],
                 "Total": fields[9],
                 "Date": '256485',
                 "Time": fields[11],
                 "Payment": fields[12],
                 "gross_income": fields[15],
                 "Rating":fields[16]
                 }
    return json_str
schema_1='Invoice_ID:STRING,Branch:STRING,City:STRING,Customer_type:STRING,Gender:STRING,Product_line:STRING,Unit_price:STRING,Quantity:STRING,Tax_5:STRING,Total:STRING,Date:STRING,Time:STRING,Payment:STRING,gross_income:STRING,Rating:STRING'


with beam.Pipeline(options=beam_options) as pipeline:
    data_ingestion=(
    pipeline
    | beam.io.ReadFromText('gs://sai_1999/supermarket_sales.csv',skip_header_lines=1)
    | beam.Map(lambda x:x.split(','))
    | "filter cash payments">> beam.Filter(lambda x:x[12]=="Cash")
    | "convert to json2">> beam.Map(to_json)
    | "write to cash table">> beam.io.WriteToBigQuery(
        tab_id,
        schema=schema_1,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
	      write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
    )

/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery.py:2102: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
